# Startup Funding Regression Project

In [1]:
!pip install fuzzywuzzy[speedup]

In [100]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from fuzzywuzzy import process, fuzz
from collections import Counter

### For this project I decided to use kaggle´s Indian Startup database due to its rawness and the feature engineering needed to make ML ready.

In [101]:
df = pd.read_csv('indian_startup.csv')
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


### With a size of 30444 rows, it gives a decent sample size for a regression ML model. Another thing to note is all the 'object' type columns.

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


### First thing I look for are the NAN percentages, I immediately spot a column that becomes useless for this project.

In [103]:
df.isna().sum() / len(df) * 100

Sr No                 0.000000
Date dd/mm/yyyy       0.000000
Startup Name          0.000000
Industry Vertical     5.617608
SubVertical          30.749014
City  Location        5.913272
Investors Name        0.788436
InvestmentnType       0.131406
Amount in USD        31.537451
Remarks              86.235217
dtype: float64

### Since there is no way of filling NANs and due to the nature of the column, I decide to drop it.

In [104]:
df = df.drop(columns='Remarks')

### Next thing I notice is that the 'Amount' column is of 'object' type, so I look into which classes are mixed there.

In [105]:
df['Amount in USD'].apply(type).value_counts()

Amount in USD
<class 'str'>      2084
<class 'float'>     960
Name: count, dtype: int64

### If we look into the column, we notice that the numbers are formatted the Indian way, so Python takes them as strings.

In [106]:
df['Amount in USD'].head(10)

0    20,00,00,000
1       80,48,394
2     1,83,58,860
3       30,00,000
4       18,00,000
5       90,00,000
6    15,00,00,000
7       60,00,000
8     7,00,00,000
9     5,00,00,000
Name: Amount in USD, dtype: object

### I decide to write a function that cleans the strings and converts them to float type.

In [107]:
def clean_amount(value):
    if isinstance(value, str):
        try:
            return float(value.replace(',', ''))
        except ValueError:
            return np.nan
    elif isinstance(value, (int, float)):
        return value
    else:
        return np.nan
    

df['Amount in USD'] = df['Amount in USD'].apply(clean_amount)

### Here I hit one of the main stonewalls in this project, the SubVertital column is too inconsistent to be a feature, due to it being more of a 'business description' value rather than a categorical subvertical.

In [108]:
df['SubVertical'].head(20)

0                                   E-learning
1                    App based shuttle service
2        Retailer of baby and toddler products
3                            Online Investment
4                  Embroiled Clothes For Women
5     Open-market, freight management platform
6                Online Food Delivery Platform
7                                     Agritech
8                                   Automobile
9                      Satellite Communication
10            Logistics Services and Solutions
11                Food Solutions For Corporate
12    Online Meat And Seafood Ordering Startup
13               Non-Banking Financial Company
14               Experience Discovery Platform
15             Real money based gaming startup
16              Online Eyewear Shopping Portal
17      Business and customer engagement tools
18             Men's Health and Wellness brand
19                                   Elearning
Name: SubVertical, dtype: object

### That incosistency makes it so there are 1942 different values in a 3044 long database, 63,7% unique values. But there is potential here to create a valuable feature.

In [109]:
df['SubVertical'].nunique()

1942

### What I decide to go for is to create a new column, which will be my feature, and fill it with 'X' to mark the uncategorized ones.

In [110]:
df['Sub-Vertical'] = 'X'

### To handle the NANs here, due to it being a SubVertical, I decide to fill in with the business´s Vertical. 

In [111]:
df['SubVertical'] = df['SubVertical'].fillna(df['Industry Vertical'])

### And here comes the 'tedious' part, I´ll manually look at the database and recognize keywords and patterns so I can create my own subset of subverticals and recategorize the startups.

In [112]:
df.loc[(df['Sub-Vertical'] == 'X') & (df['SubVertical'].str.contains('', case=False))].head(50)

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Sub-Vertical
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,2.000000e+08,X
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,8.048394e+06,X
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,1.835886e+07,X
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,3.000000e+06,X
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,1.800000e+06,X
5,6,13/01/2020,Pando,Logistics,"Open-market, freight management platform",Chennai,Chiratae Ventures,Series A,9.000000e+06,X
6,7,10/01/2020,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,Ant Financial,Private Equity Round,1.500000e+08,X
7,8,12/12/2019,Ecozen,Technology,Agritech,Pune,Sathguru Catalyzer Advisors,Series A,6.000000e+06,X
8,9,06/12/2019,CarDekho,E-Commerce,Automobile,Gurgaon,Ping An Global Voyager Fund,Series D,7.000000e+07,X
9,10,03/12/2019,Dhruva Space,Aerospace,Satellite Communication,Bengaluru,"Mumbai Angels, Ravikanth Reddy",Seed,5.000000e+07,X


### After looking at the database, I noticed 2 scenarios:
1. A set of keywords, in which case I filter by that subset of words and replace with the new category.
2. Descriptions were there is no keyword, but the context itself gives away which category it falls into, in this case I manually recategorize each one using its index.

In [113]:
def classify(rows, label):
    df.loc[rows, 'Sub-Vertical'] = label

df.loc[df['SubVertical'].str.contains('food and beverages', case=False, na=False), 'Sub-Vertical'] = 'Food Related Services'
classify([6,11,12,20,56,59,66,53,126,130,1856],'Food Related Services')

df.loc[df['SubVertical'].str.contains('car rent|carpool|bike rent|taxi|bicycle', case=False, na=False), 'Sub-Vertical'] = 'Transportation Rental Services'
classify([1,26,29,64,60,87,89,110,133,172,183,247,2099,621,221],'Transportation Rental Services')


classify([8,44,46,50,61,94,116,134,147,205,724,1023,2102,679,1081],'Automotive Related Services')

classify([236,692,849,1165,1180,1196,1418,1492,1518,1877,1951], 'Doorstep Delivery Services')

df.loc[df['SubVertical'].str.contains('logistic|supply chain|truck', case=False, na=False), 'Sub-Vertical'] = 'Logistic & Supply Chain Related Services'
classify([5,10,32,45,69,92,112,11,145,111,152,149,150,253], 'Logistic & Supply Chain Related Services')

df.loc[df['SubVertical'].str.contains('healthc|medic|doctor|clinic', case=False, na=False), 'Sub-Vertical'] = 'Health Related Services'
classify([21,28,71,99,117,151,154,161,237,304], 'Health Related Services')

df.loc[df['SubVertical'].str.contains('pharma', case=False, na=False), 'Sub-Vertical'] = 'Pharmacy'

df.loc[df['SubVertical'].str.contains('fitness|wellness| spa ', case=False, na=False), 'Sub-Vertical'] = 'Fitness & Wellness Related Services'
classify([18,42,93,34,102,136,222,224,2098], 'Fitness & Wellness Related Services')

df.loc[df['SubVertical'].str.contains('baby', case=False, na=False), 'Sub-Vertical'] = 'Baby Supplies'
classify([2,159], 'Baby Supplies')

df.loc[df['SubVertical'].str.contains('fashion', case=False, na=False), 'Sub-Vertical'] = 'Fashion Retail'
classify([4,72,91,105,113,144], 'Fashion Retail')

df.loc[df['SubVertical'].str.contains('eyewear', case=False, na=False), 'Sub-Vertical'] = 'Eyewear Retail'
classify([16], 'Eyewear Retail')

df.loc[df['SubVertical'].str.contains(r'(?=.*\blearning\b)(?=.*\bplatform\b)', regex=True, case=False, na=False), 'Sub-Vertical'] = 'E-learning'
df.loc[df['SubVertical'].str.contains(r'(?=.*\blearning\b)(?=.*\bapp\b)', regex=True, case=False, na=False), 'Sub-Vertical'] = 'E-learning'
df.loc[df['SubVertical'].str.contains(r'(?=.*\bkids\b)(?=.*\blearning\b)', regex=True, case=False, na=False), 'Sub-Vertical'] = 'E-learning'
classify([0,19,73,104,106,122,174,255,285,2024,2019,2010,2007,1916,1880,1868,1801,1756,1696,1601,1540,1469,1139,1005,983,920,634,483,479,441,371,370,290,285,2000], 'E-learning')

df.loc[df['SubVertical'].str.contains('tutor', regex=True, case=False, na=False), 'Sub-Vertical'] = 'Tutoring Services'

df.loc[df['SubVertical'].str.contains('education', regex=True, case=False, na=False), 'Sub-Vertical'] = 'Education'
classify([30,38,67,137,57,2064,2028,1827,1682,854,418,412,406,384,269], 'Education')

df.loc[df['SubVertical'].str.contains('game|gaming', case=False, na=False), 'Sub-Vertical'] = 'Gaming'
classify([15,24,129,158], 'Gaming')

df.loc[df['SubVertical'].str.contains('agri-|agriculture', case=False, na=False), 'Sub-Vertical'] = 'Agritech'
classify([7,22,81,95,591], 'Agritech')

df.loc[df['SubVertical'].str.contains('artificial int', case=False, regex=True, na=False), 'Sub-Vertical'] = 'AI Related'
df.loc[df['SubVertical'].str.contains(r'\bAI\b', case=False, regex=True, na=False), 'Sub-Vertical'] = 'AI Related'
classify([80,65,51,141,157,177,178,182,189,254], 'AI Related')

df.loc[df['SubVertical'].str.contains('wealth|invest|stock', case=False, na=False), 'Sub-Vertical'] = 'Investment/Wealth Management'
classify([98,55,47,3,188,220,250,246,238], 'Investment/Wealth Management')

df.loc[df['SubVertical'].str.contains('lending|loan|financ|payment', case=False, na=False), 'Sub-Vertical'] = 'Banking/Payments/Financial Services'
classify([78,77,103,86,109,85,13,23,54,31,35,36,162,163,173,175,165,184,225,251,257,1526,1296,495,698,1210,1903,2147], 'Banking/Payments/Financial Services')
classify([118,127,138,142,223,230,215,841,798,819], 'Non Banking Financial Services')

df.loc[df['SubVertical'].str.contains('ecommerce', case=False, na=False), 'Sub-Vertical'] = 'Ecommerce'
classify([131,128,124,123,49,96,27,169,176,191], 'Ecommerce')

df.loc[df['SubVertical'].str.contains('property', case=False, na=False), 'Sub-Vertical'] = 'Real Estate'
classify([125,74], 'Real Estate')

df.loc[df['SubVertical'].str.contains('travel', case=False, na=False), 'Sub-Vertical'] = 'Travel Related Services'
classify([155,121,101,170,219], 'Travel Related Services')
classify([146,97,75,2008,1316], 'Wine & Beer')

df.loc[df['SubVertical'].str.contains('food delivery|food ordering|grocery', case=False, na=False), 'Sub-Vertical'] = 'Grocery / Food Delivery Services'
classify([132,190,181,249,649], 'Grocery / Food Delivery Services')

classify([107,37,], 'Marketing')

df.loc[df['SubVertical'].str.contains('robotics', case=False, na=False), 'Sub-Vertical'] = 'Robotics'

df.loc[df['SubVertical'].str.contains('video', case=False, na=False), 'Sub-Vertical'] = 'Video Content & Streaming Platforms'

df.loc[df['SubVertical'].str.contains('insurance', case=False, na=False), 'Sub-Vertical'] = 'Insurance / InsurTech'

df.loc[df['SubVertical'].str.contains('communic', case=False, na=False), 'Sub-Vertical'] = 'Communication Platforms'

df.loc[df['SubVertical'].str.contains('media|news|entertainment', case=False, na=False), 'Sub-Vertical'] = 'Digital Media / Content'

df.loc[df['SubVertical'].str.contains('SaaS|Software', case=False, na=False),'Sub-Vertical'] = 'Software / SaaS'

df.loc[df['SubVertical'].str.contains('data analy|analy|data', case=False, na=False),'Sub-Vertical'] = 'Data Science/Analytics'

df.loc[df['SubVertical'].str.contains('spaces', case=False, na=False),'Sub-Vertical'] = 'Co-Working & Co-Living Spaces'

df.loc[df['SubVertical'].str.contains('career|job', case=False, na=False),'Sub-Vertical'] = 'Career & Recruitment Services'

df.loc[df['SubVertical'].str.contains('fund', case=False, na=False),'Sub-Vertical'] = 'Crowdfunding / Fundraising Platforms'

df.loc[df['SubVertical'].str.contains('matchm|dating', case=False, na=False),'Sub-Vertical'] = 'Dating Platforms'

df.loc[df['SubVertical'].str.contains('home', case=False, na=False),'Sub-Vertical'] = 'Home Related Services'

df.loc[df['SubVertical'].str.contains('cybersecurity|cyber security|security', case=False, na=False),'Sub-Vertical'] = 'Cybersecurity'

df.loc[df['SubVertical'].str.contains('tea ', case=False, na=False),'Sub-Vertical'] = 'Tea Sales'

df.loc[df['SubVertical'].str.contains('social network', case=False, na=False),'Sub-Vertical'] = 'Social Networks'

df.loc[df['SubVertical'].str.contains('beauty', case=False, na=False),'Sub-Vertical'] = 'Beauty Related Services'

df.loc[df['SubVertical'].str.contains('waste|recycl', case=False, na=False),'Sub-Vertical'] = 'Waste/Recycling Related Services'

df.loc[df['SubVertical'].str.contains('business', case=False, na=False),'Sub-Vertical'] = 'Business Development'

df.loc[df['SubVertical'].str.contains('diagnost', case=False, na=False),'Sub-Vertical'] = 'Diagnostics & Medical Testing'

df.loc[df['SubVertical'].str.contains('restaurant', case=False, na=False),'Sub-Vertical'] = 'Restaurant Tech & Services'

df.loc[df['SubVertical'].str.contains('pet', case=False, na=False),'Sub-Vertical'] = 'Pet Related Services'
# e books

### To help visualize progress, I created a percentage and rows counter.

In [114]:
progress = (df['Sub-Vertical'] != 'X').mean() * 100
rows = (df['Sub-Vertical'] != 'X').sum()

print(f"{progress:.1f}% done")
print(f'{rows} rows done')

53.7% done
1636 rows done


### Next, I plan to do the same with the Vertical column, but due to it having way clearer categories, I´ll wait until after running the model and seeing the scores.

In [115]:
df['Industry Vertical'].value_counts()

Industry Vertical
Consumer Internet             941
Technology                    478
eCommerce                     186
Healthcare                     70
Finance                        62
                             ... 
Startup Analytics platform      1
Mobile Food Ordering app        1
Financial Markets Software      1
Hiring Analytics platform       1
Flat rental Mobile App          1
Name: count, Length: 821, dtype: int64

### The other main wall I hit is the 'Investor Name' column. This column is a set of strings written in inconsistent form, mentioning the investors.
### 2 problems here:
1. The name of the investors is written differently (letter cases, spelling, spaced, separation symbols).
2. It is not a list, so each row is a string and we cant access the investors so the model can learn information about them.

In [116]:
df['Investors Name'].value_counts()

Investors Name
Undisclosed Investors                                                                                                                                             39
Undisclosed investors                                                                                                                                             30
Ratan Tata                                                                                                                                                        25
Indian Angel Network                                                                                                                                              23
Kalaari Capital                                                                                                                                                   16
                                                                                                                                                                

In [117]:
df['Investors Name'].nunique()

2412

### To get a clean set of investors I go with 3 steps:
1. Clean and standarize the strings so I can turn them into lists.
2. Remove redudants words like 'Capital' or 'India' that may mislead fuzzywuzzy when mapping the values.
3. Use fuzzywuzzy´s toke_set_ratio to get the list of unique investors and then map them.

In [134]:
# group|limited|ltd|inc|corp|corporation|company|co|management|

In [119]:
df['Investors Name'] = df['Investors Name'].fillna('Undisclosed')

redundant_words = r'\b(india|capital|partners?|ventures?|holdings?|advisors?)\b'

df['Investors List'] = df['Investors Name'].str.lower().str.replace(' and ',' , ').str.replace(' & ',' , ').str.replace(redundant_words,'', regex=True).str.split(',').apply(lambda x: [i.strip() for i in x])

In [120]:
all_investors = set([i for sublist in df['Investors List'] for i in sublist])

canonical_names = {}
for inv in all_investors:
    if canonical_names:  
        result = process.extractOne(inv, canonical_names.keys(), scorer=fuzz.token_set_ratio)
        if result:  
            match, score = result
            if score >= 70:
                canonical_names[inv] = match
            else:
                canonical_names[inv] = inv
        else:  
            canonical_names[inv] = inv
    else:
        canonical_names[inv] = inv  


### After using fuzzywuzzy I take a look into the results to see if it is good enough for mapping the database.

In [121]:
df1 = pd.DataFrame(canonical_names.items(), columns=['name','canon'])
df1[df1['canon'] == '']

,name,canon
0,,


### After looking at fuzzywuzzy´s result dictionary and being ok with it, I proceed to map the database´s values and running a value_counts to see the distribution.

In [122]:
df['Investors List'] = df['Investors List'].apply(lambda lst: [canonical_names[i] for i in lst])

investors = [inv for sublist in df['Investors List'] for inv in sublist]
counts = Counter(investors)
pd.DataFrame(counts.items(), columns=['Investor','Count']).sort_values('Count', ascending=False)

,Investor,Count
200,others ( the ten minute million),186
2,sequoia,123
24,accel us,95
509,undisclosed multiple investors,90
106,,79
...,...,...
17,nivesh growth fund,1
14,nptk,1
10,ravikanth reddy,1
7,sathguru catalyzer,1


### In order to not overload the features and flood the model with a high number of dummies, I take the main 30 investors and map the rest as 'Other' (mainly the ones that only appear once).

In [123]:
counts_s = pd.Series(counts)
main_investors = counts_s[counts_s > 2]

df['Investors List'] = df['Investors List'].apply(lambda inv_list: [inv if inv in main_investors else 'Other' for inv in inv_list])

In [124]:
investors = [inv for sublist in df['Investors List'] for inv in sublist]
new_counts = Counter(investors)
pd.DataFrame(new_counts.items(), columns=['Investor','Count']).sort_values('Count', ascending=False)

,Investor,Count
1,Other,1602
121,others ( the ten minute million),186
2,sequoia,123
18,accel us,95
276,undisclosed multiple investors,90
...,...,...
498,divyesh shah,3
494,ab kinnevik,3
493,khazanah nasional berhad,3
492,vishal khare,3


### Since models cannot read lists, I use a MultiLabelBinarizer to include the investor dummies in the main df.

In [125]:
mlb = MultiLabelBinarizer()

investor_dummies = mlb.fit_transform(df['Investors List'])

investor_columns = pd.DataFrame(investor_dummies, columns=mlb.classes_, index=df.index)

df = pd.concat([df, investor_columns], axis=1)


### FInally, I drop NANs before creating my testing and training sets.

In [126]:
df = df.dropna()

In [127]:
df.isna().sum() / len(df) * 100

Sr No                  0.0
Date dd/mm/yyyy        0.0
Startup Name           0.0
Industry Vertical      0.0
SubVertical            0.0
                      ... 
yournest angel fund    0.0
yourwecan              0.0
yuri milner            0.0
zishaan hayath         0.0
zodius                 0.0
Length: 550, dtype: float64

### For this regression project, our sets are as follows:
1. Features: Industry Vertical, Industry SubVertical, Investors, Number of Investors, Investing type and Location.
2. Target: Amount of USD invested in the startup.

In [128]:
df.columns

Index(['Sr No', 'Date dd/mm/yyyy', 'Startup Name', 'Industry Vertical',
       'SubVertical', 'City  Location', 'Investors Name', 'InvestmentnType',
       'Amount in USD', 'Sub-Vertical',
       ...
       'xiaomi inc', 'yatin kumar jain', 'ycombinator', 'yogesh bansal',
       'yournest', 'yournest angel fund', 'yourwecan', 'yuri milner',
       'zishaan hayath', 'zodius'],
      dtype='object', length=550)

In [130]:
X = pd.get_dummies(df[df['Sub-Vertical'] != 'X'].drop(columns=['Sr No','Date dd/mm/yyyy','Startup Name','Amount in USD','SubVertical','Investors Name','Investors List']))
y = df[df['Sub-Vertical'] != 'X']['Amount in USD']
y = np.log1p(y) # applied a log1p transformation to the target to reduce skewness and stabilize variance.

random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=random_state)

In [136]:
train = X_train.shape
test = X_test.shape

print(f'Train Set Shape: {train}')
print(f'Test Set Shape: {test}')

Train Set Shape: (901, 1010)
Test Set Shape: (226, 1010)


### I decide to go with a RandomForestRegressor through a RandomizedSearchCV to find the best parameters.

In [131]:
rf = RandomForestRegressor(random_state=random_state)

param_grid = {
    'n_estimators': [100, 150, 200],     
    'max_depth': [5, 10, 15, 20, 25],       
    'min_samples_split': [2, 5, 10, 15],          
    'min_samples_leaf': [1, 2, 4, 6],             
    'max_features': ['sqrt', 'log2', 0.7, 0.8],   
    'bootstrap': [True, False]                   
}

rfs = RandomizedSearchCV(rf, 
                        param_distributions=param_grid, 
                        cv=5, 
                        n_iter=30,
                        random_state=random_state,
                        verbose=0,
                        scoring='r2')

rfs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [5, 10, 15, 20, 25],
                                        'max_features': ['sqrt', 'log2', 0.7,
                                                         0.8],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [100, 150, 200]},
                   random_state=42, scoring='r2')

In [132]:
params = rfs.best_params_
params

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 0.7,
 'max_depth': 25,
 'bootstrap': True}

### As the main objective, I´m looking for a score of 0.7 - 0.8 at minimum and a difference of < 0.12 between train and test sets.

In [133]:
train_score = rfs.score(X_train, y_train)
test_score = rfs.score(X_test, y_test)

print(f'TRAIN SCORE: {train_score:.2f}')
print(f'TEST SCORE: {test_score:.2f}')

TRAIN SCORE: 0.77
TEST SCORE: 0.58


### Finally I decided to take a look into the results of the predictions.

In [136]:
y_pred = rfs.predict(X_test)
true_y_pred = np.expm1(y_pred)

results = pd.DataFrame({
    'Actual': np.expm1(y_test),
    'Predicted': true_y_pred
})

results['Error'] = results['Actual'] - results['Predicted']
results['Actual'] = results['Actual'].map('{:,.0f}'.format)
results['Predicted'] = results['Predicted'].map('{:,.0f}'.format)
results['Error'] = results['Error'].map('{:,.0f}'.format)

results

,Actual,Predicted,Error
86,"52,000,000","6,042,442","45,957,558"
586,"250,000,000","5,255,493","244,744,507"
423,"50,000,000","5,226,847","44,773,153"
772,"5,000,000","5,261,726","-261,726"
1859,"1,000,000","244,615","755,385"
...,...,...,...
533,"2,000,000","5,192,332","-3,192,332"
687,"3,000,000","5,261,726","-2,261,726"
419,"7,400,000","7,825,210","-425,210"
1682,"1,000,000","5,213,489","-4,213,489"
